**извлечение признаков из текста на естественном языке**

токенизация и очистка текста


Евгений Борисов borisov.e@solarl.ru

## библиотеки

In [1]:
import numpy as np
np.set_printoptions(precision=2) # вывод на печать до 2 знака
import pandas as pd
import re
import gzip

In [2]:
pd.options.display.max_colwidth = 200  

## загружаем тексты

In [4]:
# загружаем тексты
df = pd.read_pickle('../data/text/news.pkl.gz')
print('записей:',len(df))

записей: 3196


In [5]:
df.sample(5)

,text,tag
1674,"Экс-президент СССР Михаил Горбачев заявил, что в границах Союза может появиться новое объединение, в которое добровольно войдут бывшие советские республики.\n\nОн отметил, что в качестве возврата ...",politics
2337,"Ростов-на-Дону, 13 декабря. DON24.RU. Вчера в 20.20 на 1073 км автодороги М-4 «Дон» столкнулись две фуры, сообщает пресс-служба ГУ МВД России по Ростовской области.\n\nПо данным ведомства, 41-летн...",incident
394,Еще одна P2P-сеть закрылась В Америке по инициативе Ассоциации\nзвукозаписывающей индустрии Америки (Recording Industry Association of\nAmerica - RIAA) закрылась популярная студенческая файлообмен...,culture
86,"Устанавливаем Solaris 10 в VMware Workstation 5.5В PDF документе "" How\nto Install Solaris 10 Inside VMware Workstation 5.5 "" (размер 500Кб) по\nшагам и со скриншотами описан процесс конфигурирова...",tech
1628,"На конгрессе Всемирного боксерского совета (WBC) в Голливуде (США) Виталий Кличко был удостоен титула ""вечный чемпион мира в тяжёлом весе"". Награду нынешнему мэру Киева вручил Рой Джонс-младший.\n...",sport


## токенизация и очистка

In [7]:
# with gzip.open('../data/text/stop-nltk.txt.gz','rt',encoding='utf-8') as f: 
#     stopwords = set([ w.strip() for w in  f.read().split() if w.strip() ] )

# print('количество стоп-слов:',len(stopwords))
# sorted(stopwords)

# from Stemmer import Stemmer
# pacman -S python-pystemmer
# pip install pystemmer

In [8]:
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords as nltk_stopwords

stopwords = set(nltk_stopwords.words('russian') )

In [9]:
%%time 

df['text_clean'] = df['text'].str.lower() # приведение в lowercase

# замена символов-разделителей (-,_) на пробел
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'\W', ' ', s))
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'_', ' ', s))

# замена цифр
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'\b\d+\b', ' ', s))

# делим строки на слова (токенизация)
df['text_clean'] = df['text_clean'].apply(lambda t: [ w.strip() for w in t.split() if len(w.strip())>2 ] )

# удаление лишних слов 
df['text_clean'] = df['text_clean'].apply(lambda t:[w for w in t if w not in stopwords])

# стемминг, выделение основы слова
# df['text_clean'] = df['text_clean'].apply( lambda t: Stemmer('russian').stemWords(t) )
df['text_clean'] = df['text_clean'].apply(lambda t:[ SnowballStemmer('russian').stem(w) for w in t if w])


CPU times: user 19.9 s, sys: 30 ms, total: 20 s
Wall time: 20 s


In [10]:
df[['text_clean']].sample(3)

,text_clean
1497,"[батьківщині, вважают, неприпустим, переговор, терорист, їхніми, ватажк, йдет, заяві, батьківщин, щод, інформації, таємн, зустріч, надії, савченк, лідер, лнр, днр, розповсюдженої, змі, посилан, дж..."
754,"[цел, эл, милиц, ищет, мужчин, котор, застрел, собак, глаз, хозяйк, декабр, tut, пустыр, мкад, охотник, цел, женщин, застрел, собак, мужчин, ищет, милиц, врач, сказа, хозяйк, собак, умерл, минут, ..."
2652,"[пришл, опрос, тыс, человек, автор, фот, nagada, американск, учен, установ, мужчин, женщин, разн, подход, выбор, партнер, сообща, порта, medikforum, исследован, провел, сотрудник, калифорнийск, ун..."
